### <strong>期末專題：雙變量函數的繪圖 & 多變量函數的參數估計與極值計算</strong>
學號：411073088

姓名：陳敬翰
<hr>

 <font color=yellow>作品目標</font>：


<hr>

函數 $f(x_1,x_2) = (x_1-2)^4 + (x_1-2)^2x_2^2 + (x_2+1)^2$
繪製立體圖（wireframe）與等高線圖（contour）。

計算 \displaystyle\min_{x_1, x_2} f(x_1, x_2).